In [39]:
from pydantic import Field, BaseModel
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"


In [40]:
class Product(BaseModel):
    name:str=Field(description="Provide actual name of the product")
    # details: str = Field(description="Provide high level summary of the product")
    price: int = Field(description="provide price of the product in USD")
    

In [41]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1196df440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1196f3560>, model_name='qwen-qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [42]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    [
    ("system", "You are a helpful assistant. provide details for asked products"),
    ("human","{product}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['product'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. provide details for asked products'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], input_types={}, partial_variables={}, template='{product}'), additional_kwargs={})])

In [ ]:
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser,PydanticOutputParser
# output_parser=PydanticOutputParser(pydantic_object=Product)
output_parser=JsonOutputParser(pydantic_object=Product)

prompt=PromptTemplate(
    template="Answer the user query \n {format_instruction}\n {query}\n ",
    input_variables=["query"],
    partial_variables={"format_instruction":output_parser.get_format_instructions()},
)


In [44]:
chain=prompt|model|output_parser

In [45]:
print(chain.invoke({"product":"tell me about iPhone 15"}))


OutputParserException: Invalid json output: <think>
Okay, the user is asking about the iPhone 15. Let me start by recalling what I know. The iPhone 15 was released in September 2023, right? It's part of the 15 series, which includes models like the 15, 15 Plus, 15 Pro, and 15 Ultra. I should mention the key features.

First, the design. I remember that Apple changed the glass back to aluminum for the standard models, except maybe the Pro versions. The Dynamic Island is there, which replaces the notch. Wait, the Dynamic Island was introduced in the Pro models in the 14 series, so probably carried over to 15. Also, the buttons: the volume button became a clickable component, similar to the power button. That's a design change from the previous static buttons.

Display: The standard models (15 and 15 Plus) have an OLED display, maybe 6.1 or 6.7 inches? The Pro models have the Always-On display. The user might want to know about screen resolution and brightness.

Camera system: The iPhone 